<a href="https://colab.research.google.com/github/kanika1709/Finetuning_LLM_Llama2/blob/main/LLama_2_samsum_summarization_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers transformers[torch] datasets bitsandbytes evaluate rouge_score peft tensorboard accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:

In [2]:
#!pip install -q torch==2.0.1 transformers==4.32.1 datasets==2.14.4 peft==0.5.0 bitsandbytes==0.41.1 trl==0.7.1 wandb

In [3]:
#!pip install accelerate

In [4]:
!pip install py7zr trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 11.3 MB/s eta 0:00:00


In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import json
import re
import os
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login, login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

In [4]:
dataset = load_dataset("samsum")
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
DEFAULT_SYSTEM_PROMPT = """
Below is the text. Write a summary of the text.
""".strip()

def generate_text(data_point, document_col, summary_col, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return {
        "text_document": data_point[document_col],
        "text_summary": data_point[summary_col],
        "text": f"""### Instruction: {system_prompt}
                    ### Input:{data_point[document_col].strip()}
                     ### Response:{data_point[summary_col].strip()}
                  """.strip()
        #"text": generate_training_prompt(data_point[document_col], data_point[summary_col]),
    }

In [6]:
example = generate_text(dataset["train"][0], "dialogue","summary")

In [7]:
example

{'text_document': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'text_summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'text': "### Instruction: Below is the text. Write a summary of the text.\n                    ### Input:Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\n                     ### Response:Amanda baked cookies and will bring Jerry some tomorrow."}

In [8]:
preprocessed_dataset = dataset.map(lambda x: generate_text(data_point = x, document_col = "dialogue", summary_col = "summary"), remove_columns=[i for i in dataset["train"].features if i not in \
                                        ['text_document', 'text_summary', 'text']])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
preprocessed_dataset['test'][1]

{'text_document': "Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob: Hahaha! No one talks to the machine like that!\r\nEric: Is this his only stand-up?\r\nRob: Idk. I'll check.\r\nEric: Sure.\r\nRob: Turns out no! There are some of his stand-ups on youtube.\r\nEric: Gr8! I'll watch them now!\r\nRob: Me too!\r\nEric: MACHINE!\r\nRob: MACHINE!\r\nEric: TTYL?\r\nRob: Sure :)",
 'text_summary': 'Eric and Rob are going to watch a stand-up on youtube.',
 'text': "### Instruction: Below is the text. Write a summary of the text.\n                    ### Input:Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob: Hahaha! No one talks to the machine like that!\r\nEric: Is this his only stand-up?\r\nRob: Idk. I'll check.\r\nEric: Sure.\r

In [12]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [13]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
#Model Quantation configs
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [15]:
# PEFT Configurations

lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
OUTPUT_DIR = "samsum-summarization-llama-2-finetuned"

In [18]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=7,
    evaluation_strategy="epoch",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="wandb",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    load_best_model_at_end=True
)


In [19]:
preprocessed_dataset["train"] = preprocessed_dataset["train"].filter(lambda _, idx: idx < 100, with_indices=True)
preprocessed_dataset["test"] = preprocessed_dataset["test"].filter(lambda _, idx: idx < 20, with_indices=True)
preprocessed_dataset["validation"] = preprocessed_dataset["validation"].filter(lambda _, idx: idx < 20, with_indices=True)

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [20]:
preprocessed_dataset

DatasetDict({
    train: Dataset({
        features: ['text_document', 'text_summary', 'text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text_document', 'text_summary', 'text'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['text_document', 'text_summary', 'text'],
        num_rows: 20
    })
})

In [21]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=e4ff286bb306d98560baf66f5fc9ea79c7c3e1904962b286b81d9229020b8582
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=preprocessed_dataset["train"],
    eval_dataset=preprocessed_dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=50,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [23]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,3.437500,3.525630
1,2.891100,2.815102
2,2.152600,2.127604
4,1.720000,1.699492
5,1.483000,1.679217
6,1.634400,1.676669


TrainOutput(global_step=42, training_loss=2.334192304384141, metrics={'train_runtime': 269.1621, 'train_samples_per_second': 2.601, 'train_steps_per_second': 0.156, 'total_flos': 1333731616358400.0, 'train_loss': 2.334192304384141, 'epoch': 6.72})

In [24]:
def generate_prompt(
    document: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{document.strip()}

### Response:
""".strip()

In [26]:
preprocessed_dataset["test"]

Dataset({
    features: ['text_document', 'text_summary', 'text'],
    num_rows: 20
})

In [27]:
test_df = pd.DataFrame(preprocessed_dataset['test'])
test_df

,text_document,text_summary,text
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...,### Instruction: Below is the text. Write a su...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...,### Instruction: Below is the text. Write a su...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...,### Instruction: Below is the text. Write a su...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...,### Instruction: Below is the text. Write a su...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....,### Instruction: Below is the text. Write a su...
5,"Benjamin: Hey guys, what are we doing with the...",Hilary has the keys to the apartment. Benjamin...,### Instruction: Below is the text. Write a su...
6,Max: Know any good sites to buy clothes from?\...,Payton provides Max with websites selling clot...,### Instruction: Below is the text. Write a su...
7,Rita: I'm so bloody tired. Falling asleep at w...,Rita and Tina are bored at work and have still...,### Instruction: Below is the text. Write a su...
8,"Beatrice: I am in town, shopping. They have ni...","Beatrice wants to buy Leo a scarf, but he does...",### Instruction: Below is the text. Write a su...
9,Ivan: hey eric\r\nEric: yeah man\r\nIvan: so y...,Eric doesn't know if his parents let him go to...,### Instruction: Below is the text. Write a su...


In [28]:
example = test_df.iloc[0]

In [37]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [35]:
fine_tuned_model = trainer.model

In [38]:
summary_org = summarize(fine_tuned_model, example.text)

In [39]:
summary_org

"Hannah doesn't know Larry well and doesn't want to text him. Amanda suggests texting him. Hannah doesn't want to text him but agrees to text him."

In [41]:
example

text_document    Hannah: Hey, do you have Betty's number?\nAman...
text_summary     Hannah needs Betty's number but Amanda doesn't...
text             ### Instruction: Below is the text. Write a su...
Name: 0, dtype: object

In [42]:
example.text_summary

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [43]:
example['text_document']

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [45]:
import evaluate
predictions, references = [] , []
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(test_df.shape[0]):
    output_text = summarize(fine_tuned_model, test_df['text'][i])
    #output = model.generate(input_ids=input_ids, max_length = max_target_length)
            # convert the tokens to text
            #input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            #output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(output_text)
    references.append(test_df['text_summary'][i])
        # Metric
metric = evaluate.load("rouge")
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")
        #return rogue


Rogue1: 16.286482%
rouge2: 5.299802%
rougeL: 12.217775%
rougeLsum: 12.265926%
